# Preparação

## Importação das Bibliotecas

In [1]:
import numpy as np
import plotly.graph_objects as go
import requests
import tarfile
import os
import random
from collections import defaultdict

## Obtenção das Instâncias

In [2]:
# Função para baixar e extrair arquivos
def download_and_extract(url):
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        file_name = url.split('/')[-1]
        with open(file_name, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)
        with tarfile.open(file_name, "r:gz") as tar:
            tar.extractall()
        os.remove(file_name)
    else:
        raise Exception(f"Failed to download file from {url}")

# URL das instâncias de polígonos von Koch aleatórios (RvK)
instances_url = 'https://www.ic.unicamp.br/~cid/Problem-instances/Art-Gallery/AGPVG/agp2009a-instances/agp2009a-simplerand.tgz'

# Baixar e extrair os arquivos
download_and_extract(instances_url)

## Funções Básicas Geométricas

In [3]:
# Função para ler os vértices do polígono a partir de um arquivo no formato especificado
def read_polygon(arquivo):
    with open(arquivo, 'r') as file:
        line = file.readline().strip().split()
        num_vertices = int(line[0])
        vertices = []
        for i in range(1, len(line), 2):
            x = eval(line[i])
            y = eval(line[i + 1])
            vertices.append((x, y))
    return vertices

# Função para verificar se duas arestas se cruzam
def are_edges_intersecting(p1, p2, p3, p4):
    def orientation(p, q, r):
        val = (q[1] - p[1]) * (r[0] - q[0]) - (q[0] - p[0]) * (r[1] - q[1])
        if val == 0: return 0
        return 1 if val > 0 else 2

    o1 = orientation(p1, p2, p3)
    o2 = orientation(p1, p2, p4)
    o3 = orientation(p3, p4, p1)
    o4 = orientation(p3, p4, p2)

    if o1 != o2 and o3 != o4:
        return True

    if o1 == 0 and on_segment(p1, p3, p2): return True
    if o2 == 0 and on_segment(p1, p4, p2): return True
    if o3 == 0 and on_segment(p3, p1, p4): return True
    if o4 == 0 and on_segment(p3, p2, p4): return True

    return False

def on_segment(p, q, r):
    if (q[0] <= max(p[0], r[0]) and q[0] >= min(p[0], r[0]) and
        q[1] <= max(p[1], r[1]) and q[1] >= min(p[1], r[1])):
        return True
    return False

# Função para verificar se o polígono é simples
def is_simple_polygon(vertices):
    n = len(vertices)
    for i in range(n):
        for j in range(i + 1, n):
            p1, p2 = vertices[i], vertices[(i + 1) % n]
            p3, p4 = vertices[j], vertices[(j + 1) % n]
            if i != j and (i + 1) % n != j and i != (j + 1) % n:
                if are_edges_intersecting(p1, p2, p3, p4):
                    return False
    return True

# Função para detectar orelhas
def is_convex(p1, p2, p3):
    return (p2[0] - p1[0]) * (p3[1] - p1[1]) - (p2[1] - p1[1]) * (p3[0] - p1[0]) > 0

def is_ear(p1, p2, p3, vertices):
    if not is_convex(p1, p2, p3):
        return False
    for p in vertices:
        if p != p1 and p != p2 and p != p3 and is_point_in_triangle(p, p1, p2, p3):
            return False
    return True

def is_point_in_triangle(p, p1, p2, p3):
    def sign(p1, p2, p3):
        return (p1[0] - p3[0]) * (p2[1] - p3[1]) - (p2[0] - p3[0]) * (p1[1] - p3[1])

    d1 = sign(p, p1, p2)
    d2 = sign(p, p2, p3)
    d3 = sign(p, p3, p1)
    has_neg = (d1 < 0) or (d2 < 0) or (d3 < 0)
    has_pos = (d1 > 0) or (d2 > 0) or (d3 > 0)

    return not (has_neg and has_pos)

## Função para Visualização dos Triângulos

In [10]:
def triangulate(vertices):
    if not is_simple_polygon(vertices):
        raise Exception("O polígono não é simples (as arestas se cruzam).")

    triangles = []
    vertices = vertices.copy()
    frames = []

    while len(vertices) > 3:
        ear_found = False
        for i in range(len(vertices)):
            p1 = vertices[i]
            p2 = vertices[(i + 1) % len(vertices)]
            p3 = vertices[(i + 2) % len(vertices)]
            if is_ear(p1, p2, p3, vertices):
                triangles.append((p1, p2, p3))
                vertices.pop((i + 1) % len(vertices))
                ear_found = True
                break
        if not ear_found:
            raise Exception("Nenhuma orelha encontrada. O polígono pode ser inválido.")

        # Adicionar frame atual à lista de frames
        frame_vertices = vertices + [vertices[0]]
        triangles_x = []
        triangles_y = []
        for triangle in triangles[:-1]:  # Exclude the last triangle for now
            tx, ty = zip(*triangle)
            triangles_x.extend(tx + (None,))
            triangles_y.extend(ty + (None,))

        last_triangle_x, last_triangle_y = zip(*triangles[-1])  # Add last triangle separately
        last_triangle_x = list(last_triangle_x) + [None]
        last_triangle_y = list(last_triangle_y) + [None]

        frame_data = [
            go.Scatter(x=[v[0] for v in frame_vertices], y=[v[1] for v in frame_vertices], mode='lines+markers', name="Polígono"),
            go.Scatter(x=triangles_x, y=triangles_y, mode='lines+markers', fill='toself', name="Triângulos"),
            go.Scatter(x=last_triangle_x, y=last_triangle_y, mode='lines+markers', fill='toself', name="Último Triângulo", line=dict(color='blue'))
        ]

        frames.append(go.Frame(data=frame_data, name=str(len(frames) + 1)))

    # Adicionar o último triângulo no penúltimo frame
    if len(vertices) == 3:
        triangles.append((vertices[0], vertices[1], vertices[2]))
        tx, ty = zip(*triangles[-1])
        triangles_x.extend(tx + (None,))
        triangles_y.extend(ty + (None,))
        frame_data = [
            go.Scatter(x=[v[0] for v in vertices + [vertices[0]]], y=[v[1] for v in vertices + [vertices[0]]], mode='lines+markers', name="Polígono"),
            go.Scatter(x=triangles_x, y=triangles_y, mode='lines+markers', fill='toself', name="Triângulos")
        ]
        frames.append(go.Frame(data=frame_data, name=str(len(frames) + 1)))

    # Colorir os vértices minimizando o número de cores
    vertex_colors = color_vertices(vertices, triangles)

    guard_vertices = select_guard_vertices(vertices, vertex_colors)

    return triangles, frames, guard_vertices, vertex_colors

## Função para Visualização dos Vértices

In [11]:
def plot_triangles(vertices, frames, guard_vertices, uncolored_vertices, vertex_colors, width=800, height=600):
    fig = go.Figure(
        data=frames[0].data,
        frames=frames
    )

    total_triangles = len(vertices) - 2  # Total de triângulos formados
    num_guards = len(guard_vertices)  # Total de Guardas Principais necessárias
    num_uncolored = len(uncolored_vertices)  # Total de Guardas Auxiliares

    fig.update_layout(
        updatemenus=[dict(
            type="buttons",
            showactive=False,
            buttons=[
                dict(
                    label="Play",
                    method="animate",
                    args=[None, {"frame": {"duration": 1000, "redraw": True}, "fromcurrent": True, "mode": "immediate"}]
                ),
                dict(
                    label="Pause",
                    method="animate",
                    args=[[None], {"frame": {"duration": 0, "redraw": True}, "mode": "immediate"}]
                )
            ],
            x=0.025,
            xanchor="left",
            y=-0.05,
            yanchor="top"
        )],
        sliders=[{
            "steps": [{"args": [[f.name], {"frame": {"duration": 0, "redraw": True}, "mode": "immediate"}],
                       "label": str(i + 1),  # Ajustar para começar de 1
                       "method": "animate"} for i, f in enumerate(frames)],
            "x": 0.1,
            "len": 0.9,
            "xanchor": "left",
            "y": 0,
            "yanchor": "top",
            "currentvalue": {
                "prefix": "Triângulos: ",
                "font": {"size": 20},
                "visible": True,
                "xanchor": "left"
            },
            "transition": {"duration": 0, "easing": "cubic-in-out"},
            "pad": {"b": 10, "t": 10}  # Ajuste do preenchimento
        }],
        width=width,
        height=height,
        showlegend=False,
        margin=dict(l=10, r=10, t=100, b=50)  # Ajuste das margens para adicionar mais espaço no topo
    )

    # Adicionar cores aos vértices
    for vertex, color in vertex_colors.items():
        fig.add_trace(go.Scatter(
            x=[vertex[0]], y=[vertex[1]],
            mode='markers',
            marker=dict(color=color, size=10),
            name=f'Vertice {vertex}'
        ))

    # Adicionar vértices dos Guardas Principais
    for guard in guard_vertices:
        fig.add_trace(go.Scatter(
            x=[guard[0]], y=[guard[1]],
            mode='markers',
            marker=dict(color='black', size=15, symbol='star'),
            name=f'Guarda Principal {guard}'
        ))

    # Adicionar vértices dos Guardas Auxiliares
    for vertex in uncolored_vertices:
        fig.add_trace(go.Scatter(
            x=[vertex[0]], y=[vertex[1]],
            mode='markers',
            marker=dict(color='gold', size=15, symbol='circle'),
            name=f'Guarda Auxiliar {vertex}'
        ))

    # Atualizar layout para mostrar a quantidade de Guardas Principais e Guardas Auxiliares
    fig.update_layout(
        title=dict(
            text=f'<b>Total de Triângulos:</b> <b>{total_triangles}</b><br>'
                 f'<b>Guardas Principais:</b> <b>{num_guards}</b><br>'
                 f'<b>Guardas Auxiliares:</b> <b>{num_uncolored}</b>',
            y=0.95  # Ajuste o valor de 'y' conforme necessário para elevar o título
        )
    )

    fig.show()


## Funções para Escolher Polígono

In [6]:
def listar_quantidades_vertices():
    # Lista todos os arquivos no diretório de instâncias
    instance_files = os.listdir('agp2009a-simplerand')

    # Extrai as quantidades de vértices únicas e ordena
    vertex_counts = sorted(set(int(file.split('-')[1]) for file in instance_files))

    # Separar as quantidades de vértices em três categorias
    vertex_counts_less_100 = [count for count in vertex_counts if count < 100]
    vertex_counts_100_999 = [count for count in vertex_counts if 100 <= count < 1000]
    vertex_counts_1000_or_more = [count for count in vertex_counts if count >= 1000]

    # Função para exibir os valores
    def print_columns(lst):
        print(" ".join([str(num) for num in lst]))

    # Exibir as quantidades de vértices
    print("Quantidades de vértices disponíveis:")
    print_columns(vertex_counts_less_100)
    print_columns(vertex_counts_100_999)
    print_columns(vertex_counts_1000_or_more)

def filtrar_instancias_por_vertices():
    # Lista todos os arquivos no diretório de instâncias
    instance_files = os.listdir('agp2009a-simplerand')

    # Solicita ao usuário para inserir a quantidade exata de vértices do polígono
    exact_vertices = int(input("\nInsira a quantidade exata de vértices do polígono: "))

    # Filtra os arquivos com base na quantidade exata de vértices
    filtered_instance_files = [file for file in instance_files if int(file.split('-')[1]) == exact_vertices]
    filtered_instance_files = sorted(filtered_instance_files)  # Ordena os arquivos para exibição organizada

    if not filtered_instance_files:
        print(f"\nNenhuma instância encontrada com {exact_vertices} vértices.")
        return []

    return filtered_instance_files

def selecionar_e_carregar_instancia(filtered_instance_files):
    if not filtered_instance_files:
        return None

    # Solicita ao usuário para selecionar uma instância pelo índice
    instance_number = input("\nInsira o número da instância desejada (de 1 a 30): ").strip()

    if not instance_number.isdigit() or not (1 <= int(instance_number) <= len(filtered_instance_files)):
        raise ValueError("Número da instância inválido.")

    selected_instance_file = filtered_instance_files[int(instance_number) - 1]

    instance_path = os.path.join('agp2009a-simplerand', selected_instance_file)

    # Ler os vértices do polígono a partir do arquivo selecionado
    vertices = read_polygon(instance_path)

    return vertices


## Obtenção dos Guardas com Grafo de Adjacência e 3-Coloração

In [7]:
# Função para construir o grafo de adjacência a partir dos triângulos
def build_graph(vertices, triangles):
    graph = {tuple(v): set() for v in vertices}
    for t in triangles:
        for i in range(3):
            if tuple(t[i]) not in graph:
                graph[tuple(t[i])] = set()
            if tuple(t[(i + 1) % 3]) not in graph:
                graph[tuple(t[(i + 1) % 3])] = set()
            if tuple(t[(i + 2) % 3]) not in graph:
                graph[tuple(t[(i + 2) % 3])] = set()
            graph[tuple(t[i])].add(tuple(t[(i + 1) % 3]))
            graph[tuple(t[i])].add(tuple(t[(i + 2) % 3]))
    return graph

# Função para colorir os vértices minimizando o número de cores
def color_vertices(vertices, triangles):
    graph = build_graph(vertices, triangles)
    colors = {}
    available_colors = ['red', 'green', 'blue']

    for vertex in graph:
        neighbor_colors = {colors[neighbor] for neighbor in graph[vertex] if neighbor in colors}
        for color in available_colors:
            if color not in neighbor_colors:
                colors[vertex] = color
                break

    # Garantir que todos os vértices estão coloridos
    for vertex in vertices:
        if tuple(vertex) not in colors:
            neighbor_colors = {colors[neighbor] for neighbor in graph[tuple(vertex)] if neighbor in colors}
            for color in available_colors:
                if color not in neighbor_colors:
                    colors[tuple(vertex)] = color
                    break

    # Adicionando ponto de verificação para depuração
    for vertex in vertices:
        if tuple(vertex) not in colors:
            print(f"Vértice não colorido: {vertex}")
            print(f"Vizinhos: {graph[tuple(vertex)]}")

    return colors

def select_guard_vertices(vertices, colors):
    color_count = defaultdict(int)
    for color in colors.values():
        color_count[color] += 1

    min_color = min(color_count, key=color_count.get)

    guards = [vertex for vertex, color in colors.items() if color == min_color]
    uncolored_vertices = [vertex for vertex in vertices if tuple(vertex) not in colors]

    return guards, uncolored_vertices

# Escolher Polígono

In [18]:
# Listar as quantidades de vértices disponíveis
listar_quantidades_vertices()

# Filtrar e listar as instâncias com base na quantidade máxima de vértices
filtered_instance_files = filtrar_instancias_por_vertices()

# Selecionar e carregar a instância escolhida
vertices = selecionar_e_carregar_instancia(filtered_instance_files)

Quantidades de vértices disponíveis:
20 40 60 80
100 200 300 400 500 600 700 800 900
1000 1250 1500 1750 2000 2250 2500

Insira a quantidade exata de vértices do polígono: 20

Insira o número da instância desejada (de 1 a 30): 10


# Visualizar Solução

In [19]:
# Executar a triangulação com visualização passo a passo
triangles, frames, guard_vertices, vertex_colors = triangulate(vertices)

# Selecionar os vértices que representam as câmeras
guard_vertices, uncolored_vertices = select_guard_vertices(vertices, vertex_colors)

# Plotar os triângulos resultantes
plot_triangles(vertices, frames, guard_vertices, uncolored_vertices, vertex_colors, width=980, height=700)